In [1]:
import sys
!{sys.executable} -m pip install pycbc lalsuite ligo-common --no-cache-dir

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [ ]:
%matplotlib inline
from pycbc.waveform import get_td_waveform
from pycbc.filter import match, overlap
from pycbc.psd import aLIGOZeroDetHighPower
import pylab
import matplotlib.pyplot as plt
import numpy as np

# Function GenTemplate: apx string, mass1 float, mass 2 float, f_low float, sample_rate int; 
# returns a 1D array of two polarized waves from the chosen template.

# Make a 30*30 grid of IMRPhenomD waveforms
def GenTemplate (mass1, mass2, apx="IMRPhenomD", f_low=20, sample_rate=4096):
    polarized_waves = np.empty(2, object)
    hp,hc = get_td_waveform(approximant=apx,
                           mass1=mass1,
                           mass2=mass2,
                           f_lower=f_low,
                           delta_t=1.0/sample_rate)
    polarized_waves[0]=hp
    polarized_waves[1]=hc
    return polarized_waves

masses2=masses1= np.linspace(10., 30., num=30)

# Make my bank of templates 0
templates_c0=templates_p0=np.empty([len(masses1),len(masses2)], object)
for i in range(0,len(masses1)):
    for j in range(0,len(masses2)):
        templates_p0[i][j]=GenTemplate(masses1[i],masses2[j])[0]
        templates_c0[i][j]=GenTemplate(masses1[i],masses2[j])[1]
        
def GetMatch (template0,template1,f_low=20):
    #resize two templates
    tlen = max(len(template0),len(template1))
    template1.resize(tlen)
    template0=template0.copy()
    template0.resize(tlen)
    
    #generate psd
    delta_f = 1.0/template1.duration
    flen = tlen/2 +1
    psd = aLIGOZeroDetHighPower(flen,delta_f,f_low)
    
    #calculate match
    m,i = match(template0,template1,psd=psd,low_frequency_cutoff=f_low)
    return m

def GetGlobalMatch(template1, templates0=templates_p0,f_low=20):
    local_matches = np.empty([30,30])
    for i in range(0,30):
        for j in range(0,30):
            tp0=templates0[i][j]
            local_match = GetMatch(tp0,template1)
            local_matches[i][j] = local_match
    global_match = np.amax(local_matches)
    return global_match

rnd_mass_combos = np.random.uniform(low=10.,high=30.,size=(3,2))
glb_matches=np.empty(3)
for i in range(0,3):
    tp1=GenTemplate(rnd_mass_combos[i][0],rnd_mass_combos[i][1],apx="SEOBNRv4")[0]
    glb_match = GetGlobalMatch(template1=tp1)
    glb_matches[i]=glb_match
    print (glb_match)



In [ ]:
%matplotlib inline
from pycbc.waveform import get_td_waveform
from pycbc.filter import match, overlap
from pycbc.psd import aLIGOZeroDetHighPower
import pylab
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

start=datetime.now()
print (start)
# Function GenTemplate: apx string, mass1 float, mass 2 float, f_low float, sample_rate int; 
# returns a 1D array of two polarized waves from the chosen template.

# Make a 30*30 grid of IMRPhenomD waveforms
def GenTemplate (mass1, mass2, apx="IMRPhenomD", f_low=20, sample_rate=4096):
    polarized_waves = np.empty(2, object)
    hp,hc = get_td_waveform(approximant=apx,
                           mass1=mass1,
                           mass2=mass2,
                           f_lower=f_low,
                           delta_t=1.0/sample_rate)
    polarized_waves[0]=hp
    polarized_waves[1]=hc
    return polarized_waves

masses2=masses1= np.linspace(10., 30., num=30)

# Make my bank of templates 0
templates_c0=templates_p0=np.empty([len(masses1),len(masses2)], object)
for i in range(0,len(masses1)):
    for j in range(0,len(masses2)):
        templates_p0[i][j]=GenTemplate(masses1[i],masses2[j])[0]
        templates_c0[i][j]=GenTemplate(masses1[i],masses2[j])[1]
        
def GetMatch (template0,template1,f_low=20):
    #resize two templates
    tlen = max(len(template0),len(template1))
    template1.resize(tlen)
    template0=template0.copy()
    template0.resize(tlen)
    
    #generate psd
    delta_f = 1.0/template1.duration
    flen = tlen/2 +1
    psd = aLIGOZeroDetHighPower(flen,delta_f,f_low)
    
    #calculate match
    m,i = match(template0,template1,psd=psd,low_frequency_cutoff=f_low)
    return m

def GetGlobalMatch(template1, templates0=templates_p0,f_low=20):
    local_matches = np.empty([30,30])
    for i in range(0,30):  
        for j in range(0,30):
            tp0=templates0[i][j]
            local_match = GetMatch(tp0,template1)
            local_matches[i][j] = local_match
    global_match = np.amax(local_matches)
    return global_match

mass_combo_2=mass_combo_1 = np.linspace(10.,30.,num=20)
glb_matches=np.empty([20,20])
for i in range(0,20):
    for j in range(0,20):
        this_m1 = mass_combo_1[i]
        this_m2 = mass_combo_2[j]
        tp1=GenTemplate(this_m1,this_m2,apx="SEOBNRv4")[0]
        glb_match = GetGlobalMatch(template1=tp1)
        glb_matches[i]=glb_match
        print (glb_match)
        print (datetime.now()-start)
        
#print(glb_matches)


plt.figure()
cp = plt.contourf(mass_combo_1,mass_combo_2,glb_matches)
plt.colorbar(cp)
plt.xlabel('masses 1')
plt.ylabel('masses 2')
plt.show()
print (datetime.now()-start)

2019-06-20 14:56:08.260431
0.9893804831540857
0:00:34.005403
0.5784397775100735
0:00:52.488465
0.9773339555034971
0:01:10.895825
0.7121673374920151
0:01:31.266306
0.9342530934682338
0:01:51.424672
0.9807533405442678
0:02:11.175913
0.921337577358767
0:02:30.523032
0.9464728569903205
0:02:48.912512
0.9483643274419814
0:03:07.959904
0.9786383200727052
0:03:32.340098
0.9579955466951466
0:03:52.832326
0.9878445677276448
0:04:13.298253
0.939039228537262
0:04:33.375709
0.995427352990306
0:04:54.449587
0.925017197516452
0:05:15.662284
0.9940567514186456
0:05:42.187437
0.9516804147536575
0:06:08.119649
0.9924703877629976
0:06:33.932047
0.9723235742710424
0:06:56.478117
0.9891947121701685
0:07:20.408747
0.5784397775100735
0:07:45.585500
0.9438228518483445
0:08:08.863741
0.9806312395712775
0:08:36.961125
0.9944359795317395
0:09:11.870208
0.9494184257317251
0:09:45.908347
0.9962768453066285
0:10:19.228690
0.9974043412052084
0:10:46.238621
0.9882302128476529
0:11:12.801545
0.9949356011034971
0:11:3

0.9845196883326638
1:28:22.965189
0.986599004253694
1:28:43.811342
0.9949924804361552
1:29:06.597356
0.9951548922804403
1:29:26.624682
0.9962936478275889
1:29:47.098942
0.9948684904506726
1:30:09.474375
0.9968635788625366
1:30:36.165477
0.9948944235711986
1:30:55.955833
0.9956040411204841
1:31:14.009080
0.9953567869309469
1:31:33.760302
0.9956765747681835
1:31:51.367439
0.9952948820726136
1:32:09.775195
0.9953385905352845
1:32:29.673878
0.9958407567777019
1:32:48.166593
0.9960026530632113
1:33:09.323241
0.9959062107125298
1:33:26.996468
0.9954773923848181
1:33:44.492587
0.9943917108370137
1:34:02.393221
0.995427352990306
1:34:33.663185
0.99712271831226
1:34:51.812523
0.9958494071272199
1:35:13.012466
0.9954338970060919
1:35:45.073960
0.9899835913803565
1:36:05.933276
0.9966493779953937
1:36:24.941390
0.9931572242132212
1:36:43.017061
0.9953097071897753
1:37:00.849122
0.9965903531157695
1:37:18.932798
0.9962225951707657
1:37:53.333161
0.9967412119970379
1:38:19.407006
0.9964134594655143

Next time: 
try printing out the pair of masses (used for template0) corresponding to the global match for each template1
also print out the pair of indices we are currently computing with
and maybe also flag strangely low matches